In [3]:
from openai import OpenAI
import os

from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
key=os.environ["OPENAI_API_KEY"]

# key =os.getenv("OPENAI_API_KEY")
api_key = str('sk-wHlFuIGVmOo1P9Co4PkXG6ZzwxL1XCwuV28SQL2xnGkJ4b1t')
api_url = "https://api.zetatechs.com/v1"
#cLient = OpenAI(base_url=api_url, api_key=api_key)
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4", base_url=api_url, api_key=api_key)

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello, Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 12, 'total_tokens': 23, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_5b26d85e12', 'id': 'chatcmpl-BJivt8pkNkKH0TScwsEo9Uab4Nt76', 'finish_reason': 'stop', 'logprobs': None}, id='run-0b1cafb2-d706-48f5-a7e8-8ac1aa077a6e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 11, 'total_tokens': 23, 'input_token_details': {}, 'output_token_details': {}})

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I can't provide or guess personal information, including your name. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_5b26d85e12', 'id': 'chatcmpl-BJivuNaN4MoVyR7deabsoakgOD8Ho', 'finish_reason': 'stop', 'logprobs': None}, id='run-d02e4536-0902-4b58-943e-8d46598d6838-0', usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37, 'input_token_details': {}, 'output_token_details': {}})

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. What can I do for you, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 35, 'total_tokens': 49, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_5b26d85e12', 'id': 'chatcmpl-BJivwSvoOD9Ikmz9cNKTuNZtuW6NF', 'finish_reason': 'stop', 'logprobs': None}, id='run-e55d7c76-c680-46ab-bec9-872b3ed0a409-0', usage_metadata={'input_tokens': 35, 'output_tokens': 14, 'total_tokens': 49, 'input_token_details': {}, 'output_token_details': {}})

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


In [8]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "abc123"}}
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello, Bob! How can I assist you today?


In [10]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. How can I help you further, Bob?


In [11]:
output

{'messages': [HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='46b0abfe-0105-4b02-a853-733179c128a7'),
  AIMessage(content='Hello, Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_5b26d85e12', 'id': 'chatcmpl-BJivz7cQz24jCEOhoU5vKKAmnC6OP', 'finish_reason': 'stop', 'logprobs': None}, id='run-0172e8ae-2f6d-4efc-994d-79fe45674c58-0', usage_metadata={'input_tokens': 13, 'output_tokens': 11, 'total_tokens': 24, 'input_token_details': {}, 'output_token_details': {}}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='8d1c2bc6-870a-44d9-a5dd-7560b66967cb'),
  AIMessage(content='Your name is Bob. How can I help you further, Bob?', additional_kwargs={'refusal': None}, response_metada

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [13]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy, Jim! Pleasure t' cross paths with ye on the high digital seas. How can I be assistin' ye today?


In [15]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yar, yer name be Jim, matey! What ventures can I aid ye with on this fine day?


In [16]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [17]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [20]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Chinese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好！我是你的助手。怎么可以帮到你，Bob？


In [21]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你的名字是Bob。还有什么我可以帮助你的吗？


In [23]:
prompt_template.invoke({"messages": input_messages, "language": language})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability in Chinese.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='ca6cfb7a-a23a-48a3-96c0-1ee4e4b5cc9a')])

In [58]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=71,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [59]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({'messages':trimmed_messages, 'language':state["language"]})
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [60]:
config = {"configurable": {"thread_id": "abc4567"}}
query = "What is my name"

input_messages = messages + [HumanMessage(query)]

app.invoke({"messages":input_messages, "language":"English"},config)["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have access to personal information like your name unless you tell me what it is.


In [61]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 is.
